### Домашка 3

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД (без датафрейма)

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)

3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [314]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [315]:
from pymongo import MongoClient
from pymongo import errors

client = MongoClient('localhost', 27017)
db = client['vacancies']

#### Код добавления вакансий в MongoDB

In [316]:
def add_one_vacancy_into_db(collection, vacancy: dict, replace_duplicates=False):
    """
    Функция добавляет одну вакансию в mongodb в указанную коллекцию
    Параметры: коллекция куда добавляем, вакансия (словарь), заменять ли дубли
    Возвращает код результата: добавлено, заменено (был дубль), пропущено (был дубль)
    """
    
    result = None #возможные значения: inserted, replaced, skipped
    
    try:
        collection.insert_one(vacancy)
        result = 'inserted'
    except errors.DuplicateKeyError: 
        
        if replace_duplicates:
            #обновляем (заменяем) дубль
            collection.replace_one({'_id' : vacancy['_id']}, vacancy) 
            result = 'replaced'
        else:
            #такая вакансия уже добавлена, пропускаем ее
            result = 'skipped'
    return result

In [318]:
def add_many_vacancies_into_db(collection, vacancies):
    """
    Функция добавляет список вакансий в mongodb. Дубли не проверяются
    Параметры: коллекция куда добавляем, список вакансий (каждая вакансия это словарь)
    """
    collection.insert_many(vacancies)

#### Код для HH.ru

In [319]:
def parse_offer_hh(offer_text):
    """функция парсит строку с  зарплатой для hh.ru"""
    
    if offer_text is None:
        return (None, None, None)
    
    offer_text = offer_text.replace(chr(160), '')
    offer_text = offer_text.replace(chr(32), '')
    offer_text = offer_text.replace('.', '')
    
    digits = '1234567890'
    
    #парсим вида 3000-4000руб
    if offer_text[0] in digits:
        sep_pos = offer_text.find('-')
        min_offer = int(offer_text[:sep_pos])
        txt = offer_text[sep_pos+1:]
        
        for i in range(0, len(txt)):
            if txt[i] not in digits:
                sep_pos = i
                break
                
        max_offer = int(txt[:sep_pos])
        currency = txt[sep_pos:]
        
        return (min_offer, max_offer, currency)

    
    #парсим вида от3000руб
    if offer_text[:2] == 'от':
        txt = offer_text[2:]  
        
        for i in range(0, len(txt)):
            if txt[i] not in digits:
                sep_pos = i
                break
                
        min_offer = int(txt[:sep_pos])
        max_offer = None
        currency = txt[sep_pos:]
        
        return (min_offer, max_offer, currency)
    
    
    #парсим вида до3000руб
    if offer_text[:2] == 'до':
        txt = offer_text[2:]  
        
        for i in range(0, len(txt)):
            if txt[i] not in digits:
                sep_pos = i
                break
                
        min_offer = None
        max_offer = int(txt[:sep_pos])
        currency = txt[sep_pos:]
        
        return (min_offer, max_offer, currency)
    
    
    #если остался вариант, который мы не учли, заносим его в currency
    return (None,None,offer_text)


In [320]:
def parse_id_from_href_hh(href):   
    """функция парсит id вакансии из ссыкли на эту вакансию для hh.ru"""
    
    tmp = href.replace('https://novosibirsk.hh.ru/vacancy/', '')
    return tmp[:tmp.find('?')]


In [321]:
def parse_hh(search_request, quick_adding=False, replace_duplicates=False):
    """
    Фукнция парсит сайт hh.ru и добавляет вакансии в базу mongodb в отдельную коллекцию hh
    Параметры:
        search_request - поисковый запрос
        quick_adding - быстрое добавление (используется insert_many) или медленное (insert_one),
                       используется для первичного заполнения пустой коллекции,
                       никак не обрабатывает дубли и выбрасывает ошибку в этом случае,
                       медленное добавление проверяет дубли и может заменять или пропускать их
        replace_duplicates - заменять дубли или нет,
                             если заменять, то вакансия заменяется новой, иначе пропускается
    """
    
    
    #подсчитаем сколько добавлено, заменено, пропущено
    results = {'inserted' : 0, 'replaced' : 0, 'skipped' : 0}
    
    #сюда сложим вакансии для быстрого добавления методом insert_many
    vacancies_dict = {}
    
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

    domain_url = 'https://novosibirsk.hh.ru'
    local_url = '/search/vacancy'
    
    params = {'area' : '113',
              'clusters' : 'true',
              'enable_snippets' : 'true',
              'text' : search_request,
              'schedule' : 'remote'
             }

    is_last_page = False
    while not is_last_page:
        
        response = requests.get(domain_url + local_url, headers=headers, params=params).text      
        soup = bs(response, 'lxml')

        vacancys_list = soup.find_all('div', {'class':'vacancy-serp-item'})

        for vacancy in vacancys_list:
            
            tittle_tag = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
            tittle = tittle_tag.getText().strip()                      
            href = tittle_tag['href']
            id = parse_id_from_href_hh(href)        
            
            company = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).getText().strip()

            offer_tag = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
            offer_str = None if offer_tag == None else offer_tag.getText()

            city = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-address'}).getText()

            
            block_date = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-date'})
            date = None if block_date == None else block_date.getText().replace('\xa0', ' ')

            offer_min, offer_max, currency = parse_offer_hh(offer_str)

            current_vacancy = {'_id' : id,
                                    'tittle' : tittle,
                                    'site': 'hh.ru',
                                    'company' : company,
                                    'offer_min' : offer_min,
                                    'offer_max' : offer_max,
                                    'currency' : currency,
                                    'href' : href,
                                    'city' : city,
                                    'date' : date
                                   }
            
            #добавляем вакансию в db (медленное добавление по одной)         
            if not quick_adding:
                result_code = add_one_vacancy_into_db(db.hh, current_vacancy, 
                                                     replace_duplicates=replace_duplicates)
                results[result_code] += 1
                
            else:
                vacancies_dict[current_vacancy['_id']] = current_vacancy

        
        next_page_tag = soup.find('a', {'data-qa':'pager-next'})
        if next_page_tag == None:
            is_last_page = True
        else:
            local_url = next_page_tag['href']
            
    if quick_adding:
        #быстрое добавление все сразу
        add_many_vacancies_into_db(db.hh, vacancies_dict.values())
        print('inserted', len(vacancies_dict))
    else:
        print(results)

#### Код для Superjob

In [322]:
def parse_offer_superjob(offer_text):
    """функция парсит строку с  зарплатой для superjob.ru"""
    
    if offer_text == 'По договорённости':
        return (None, None, None)
    
    digits = '1234567890'
    
    offer_text = offer_text.replace(chr(160), '')
    offer_text = offer_text.replace('руб./месяц', '')    

    
    #парсим вида 3000-4000
    if offer_text[0] in digits:
        sep_pos = offer_text.find('—')
        
        min_offer = int(offer_text[:sep_pos])
        max_offer = offer_text[sep_pos+1:]
        currency = 'руб'
        
        return (min_offer, max_offer, currency)
    
    
    #парсим вида от3000
    if offer_text[:2] == 'от':
        txt = offer_text[2:]  
                
        min_offer = int(txt)
        max_offer = None
        currency = 'руб'
        
        return (min_offer, max_offer, currency)
    
    
    #парсим вида до3000
    if offer_text[:2] == 'до':
        txt = offer_text[2:]  
                
        min_offer = None
        max_offer = int(txt)
        currency = 'руб'
        
        return (min_offer, max_offer, currency)
    
    
    #парсим вида 3000
    if offer_text[0] in digits:
        
        min_offer = int(offer_text)
        max_offer = int(offer_text)
        currency = 'руб'
        
        return (min_offer, max_offer, currency)
    
    
    #если остался вариант, который мы не учли, заносим его в currency
    return (None,None,offer_text)


In [323]:
def parse_id_from_href_superjob(href):   
    """функция парсит id вакансии из ссыкли на эту вакансию для superjob.ru"""
    
    tmp = href.replace('.html', '')
    return tmp[tmp.rfind('-')+1:]

In [324]:
def parse_superjob(search_link, quick_adding=False, replace_duplicates=False):
   
    """
    Фукнция парсит сайт superjob.ru и добавляет вакансии в базу mongodb в отдельную коллекцию hh
    Параметры:
        search_request - поисковый запрос
        quick_adding - быстрое добавление (используется insert_many) или медленное (insert_one),
                       используется для первичного заполнения пустой коллекции,
                       никак не обрабатывает дубли и выбрасывает ошибку в этом случае,
                       медленное добавление проверяет дубли и может заменять или пропускать их
        replace_duplicates - заменять дубли или нет,
                             если заменять, то вакансия заменяется новой, иначе пропускается
    """
    
    
    #подсчитаем сколько добавлено, заменено, пропущено
    results = {'inserted' : 0, 'replaced' : 0, 'skipped' : 0}
    
    #сюда сложим вакансии для быстрого добавления методом insert_many
    vacancies_dict = {}


    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

    domain_url = 'https://russia.superjob.ru/'
    

    is_last_page = False
    while not is_last_page:
        

        response = requests.get(domain_url + search_link, headers=headers).text    
        soup = bs(response, 'lxml')

        vacancys_list = soup.find_all('div', {'class':'_3zucV _1fMKr undefined _1NAsu'})

        for vacancy in vacancys_list:

            
            block = vacancy.find('div', {'class':'jNMYr GPKTZ _1tH7S'})
            if block == None:
                continue


            block_tittle = block.findChildren(recursive=False)[0].findChildren(recursive=False)[0]

            tittle = block_tittle.getText()
            href = block_tittle['href']
            id = parse_id_from_href_superjob(href)

            offer_str = vacancy.find('div', {'class':'jNMYr GPKTZ _1tH7S'}).findChildren(recursive=False)[1].getText()

            block_company = vacancy.find('span', {'class':'f-test-text-vacancy-item-company-name'})
            if block_company == None:
                company = None
            else:
                company =  block_company.findChildren(recursive=False)[0].getText()

            blocks_date_city = vacancy.find('span', {'class':'f-test-text-company-item-location'})\
                                                                        .findChildren(recursive=False)

            date = blocks_date_city[0].getText()
            city = blocks_date_city[2].getText()

            offer_min, offer_max, currency = parse_offer_superjob(offer_str)


            current_vacancy = {'_id' : id,
                                    'tittle' : tittle,
                                    'site': 'superjob.ru',
                                    'company' : company,
                                    'offer_min' : offer_min,
                                    'offer_max' : offer_max,
                                    'currency' : currency,
                                    'href' : href,
                                    'city' : city,
                                    'date' : date
                                   }
            
            #добавляем вакансию в db (медленное добавление по одной)         
            if not quick_adding:
                result_code = add_one_vacancy_into_db(db.superjob, current_vacancy, 
                                                     replace_duplicates=replace_duplicates)
                results[result_code] += 1
                
            else:
                vacancies_dict[current_vacancy['_id']] = current_vacancy


        
        break
        
        next_page_tag = soup.find('a', {'class':'f-test-button-dalshe'})
        if next_page_tag == None:
            is_last_page = True
        else:
            local_url = next_page_tag['href']

    if quick_adding:
        #быстрое добавление все сразу
        add_many_vacancies_into_db(db.superjob, vacancies_dict.values())
        print('inserted', len(vacancies_dict))
    else:
        print(results)


### Основной код

In [325]:
#поисковый запрос

search_request_hh = 'программист 1С'

# superjob  приводит запросы вида "программист 1С" и "data science" к виду "programmist-1s" и "data-scientist" 
# и вставляет их в хвост ссылки. В рамках домашки мы такое преобразование делать не будет
search_request_superjob_link = '/vakansii/programmist-1s.html?remote_work_binary=2&click_from=facet'


#### Первоначальное заполнение пачкой (дубли не проверяем)

hh.ru

In [326]:
%%time

parse_hh(search_request_hh, quick_adding=True)

inserted 240
Wall time: 15 s


superjob.ru

In [327]:
%%time
parse_superjob(search_request_superjob_link, quick_adding=True)

inserted 20
Wall time: 2.32 s


#### Регулярное заполнение (обновление) с заменой или обновление дублей

hh.ru

In [329]:
%%time

parse_hh(search_request_hh, quick_adding=False, replace_duplicates=True)

{'inserted': 63, 'replaced': 240, 'skipped': 0}
Wall time: 14.8 s


superjob.ru

In [330]:
%%time
parse_superjob(search_request_superjob_link, quick_adding=False, replace_duplicates=False)

{'inserted': 2, 'replaced': 0, 'skipped': 18}
Wall time: 3.03 s


#### Извлекаем вакансии из базы по условию

In [349]:
offer = 120000

vacancies = db.hh.find({'$or' : [{'offer_min': {'$gt' : offer}}, {'offer_max': {'$gt' : offer}}]})\
                 .limit(5)

for vacancy in vacancies:
    print(vacancy)

{'_id': 'https://hhcdn.ru/click', 'tittle': 'Web-разработчик (1С-Битрикс)', 'site': 'hh.ru', 'company': 'ООО Джилаб', 'offer_min': 60000, 'offer_max': 150000, 'currency': 'руб', 'href': 'https://hhcdn.ru/click?b=189269&place=35', 'city': 'Москва, Ленинский проспект', 'date': None}
{'_id': '37146832', 'tittle': 'Программист 1С', 'site': 'hh.ru', 'company': 'ЭЛИАС', 'offer_min': 130000, 'offer_max': 160000, 'currency': 'руб', 'href': 'https://novosibirsk.hh.ru/vacancy/37146832?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1', 'city': 'Москва, ВДНХ', 'date': '29 июня'}
{'_id': '37293588', 'tittle': 'Программист 1С', 'site': 'hh.ru', 'company': 'Р-Лайн (R-Line)', 'offer_min': 120000, 'offer_max': 150000, 'currency': 'руб', 'href': 'https://novosibirsk.hh.ru/vacancy/37293588?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1', 'city': 'Санкт-Петербург, Лесная', 'date': '28 июня'}
{'_id': '37739919', 'tittle': 'Ведущий програ

In [350]:
offer = 120000

vacancies = db.superjob.find({'$or' : [{'offer_min': {'$gt' : offer}}, {'offer_max': {'$gt' : offer}}]})\
                 .limit(5)

for vacancy in vacancies:
    print(vacancy)

{'_id': '33937924', 'tittle': 'Ведущий разработчик 1С ERP', 'site': 'superjob.ru', 'company': 'Первый БИТ', 'offer_min': None, 'offer_max': 200000, 'currency': 'руб', 'href': '/vakansii/veduschij-razrabotchik-1s-erp-33937924.html', 'city': 'Москва', 'date': '29 июня'}
